In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'hose_inside_diameter'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['HoseI_D_']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1)], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

5119


In [5]:
# df[df['HoseI_D_'].astype(str)!='nan']['HoseI_D_'].apply(lambda x: re.sub(r'x.*|.*\-\s?','',str(x))).explode().value_counts()

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
start=r'(?i)(^.{0,8})|()'
df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)((?<!\w)(?<!\-)(?<!x\s)(?<!\w\s)\d*.?\d+\/\d+)|()''' 
df['name'] = df['start'].apply(lambda x: re_extract(pattern, str(x)))
# df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))


pat=r'(?i)(\d*.?\d+\/\d+)|()'
df['matches']=df['HoseI_D_'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|"','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]

3622


In [8]:
names=df[(df['name'].astype(str)!='[]')]
names['name']=names['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
names['name'].explode().value_counts()
# print(len(names))

names[curation_col]=names['name']
matchname=names[['external_id',curation_col]]
matchname

external_id Q:hose_inside_diameter
1          90601               ["5/8""]
2          90600               ["5/8""]
3          90603               ["3/4""]
4          90604               ["3/4""]
5          90602               ["3/4""]
...          ...                    ...
5089   MB3309X12               ["3/4""]
5090    MB3309X4               ["1/4""]
5091    MB3309X6               ["3/8""]
5092    MB3309X8               ["1/2""]
5103       54600               ["3/8""]

[3831 rows x 2 columns]

# N/A

In [9]:
na=df[(df['name'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na)+len(matchname))

5110


In [10]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchname))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

_
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [12]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchname) 

In [13]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 